In [1]:
import json
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# ✅ 1️⃣ 학습 데이터 로드
with open("labeling_novels_filtered.json", "r", encoding="utf-8") as f:
    labeled_data = json.load(f)

In [2]:
# ✅ 2️⃣ 데이터셋 변환
train_data = [{"input": item["description"], "target": item["summary"]} for item in labeled_data]
dataset = Dataset.from_dict({
    "input": [item["input"] for item in train_data],
    "target": [item["target"] for item in train_data]
})

In [3]:
# ✅ 3️⃣ KoBART 모델 & 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
# ✅ 4️⃣ 데이터 토크나이징 (Trainer가 이해할 수 있는 형태로 변환)
def preprocess_data(example):
    model_inputs = tokenizer(
        example["input"], 
        max_length=1024, 
        truncation=True, 
        padding="max_length"
    )
    
    # 🔹 "target"을 "labels"로 변환 후 토크나이징
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target"], 
            max_length=250, 
            truncation=True, 
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [5]:
# ✅ 5️⃣ 데이터셋 변환 (batched=True로 속도 최적화)
tokenized_dataset = dataset.map(preprocess_data, batched=True, remove_columns=["input", "target"])


Map:   0%|          | 0/217 [00:00<?, ? examples/s]

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [6]:
# ✅ 6️⃣ 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart_summary_finetuned",
    evaluation_strategy="epoch",  # 🔹 매 epoch마다 평가
    save_strategy="epoch",        # 🔹 매 epoch마다 저장
    per_device_train_batch_size=2,  # 🔹 배치 크기 (GPU 메모리에 따라 조정 가능)
    per_device_eval_batch_size=2,
    num_train_epochs=5,  # 🔹 학습 횟수 (필요에 따라 조정 가능)
    weight_decay=0.01,  # 🔹 가중치 감쇠 (과적합 방지)
    save_total_limit=5,  # 🔹 체크포인트 최대 개수
    logging_dir="./logs",
    logging_steps=100
    
)

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# ✅ 1️⃣ 🤗 Datasets의 train_test_split() 사용
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# ✅ 2️⃣ train/eval 데이터셋 설정
train_data = dataset_split["train"]
eval_data = dataset_split["test"]

# ✅ 3️⃣ Trainer 객체 생성 (eval_dataset 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,  # 🔹 평가 데이터셋 추가!
    tokenizer=tokenizer
)


/var/folders/wz/cf10k4857715p70flsxw2g9m0000gn/T/ipykernel_2970/360371609.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# ✅ 7️⃣ Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,   # ✅ 훈련 데이터
    eval_dataset=eval_data,     # ✅ 평가 데이터 추가!
    tokenizer=tokenizer
)


/var/folders/wz/cf10k4857715p70flsxw2g9m0000gn/T/ipykernel_2970/2867482875.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# ✅ 8️⃣ 학습 시작 🚀
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.817668
2,2.033400,0.783941
3,0.512600,0.816674
4,0.289700,0.840684
5,0.209200,0.862921


TrainOutput(global_step=435, training_loss=0.7102061107240875, metrics={'train_runtime': 192.7244, 'train_samples_per_second': 4.488, 'train_steps_per_second': 2.257, 'total_flos': 527422036377600.0, 'train_loss': 0.7102061107240875, 'epoch': 5.0})

In [10]:
# ✅ 9️⃣ 모델 저장
model.save_pretrained("./kobart_summary_finetuned")
tokenizer.save_pretrained("./kobart_summary_finetuned")

print("✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.")

✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.


In [11]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

best_checkpoint = "./kobart_summary_finetuned/checkpoint-174"  # epoch 2 모델 체크포인트
save_path = "./kobart_best_model"

# 모델과 토크나이저 불러오기
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ 가장 좋은 모델이 '{save_path}'에 저장되었습니다!")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ 가장 좋은 모델이 './kobart_best_model'에 저장되었습니다!


In [12]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# ✅ 저장된 모델 불러오기
best_checkpoint = "./kobart_best_model"
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# ✅ 테스트할 입력 문장
test_text = "이상주의자인 스노우볼과 음흉한 현실주의자인 나폴레온의 싸움은 당연히 나폴레온의 승리로 끝나고 만다.\n 나폴레온의 독재가 시작되었고, 이제는 철통 같은 규율만을 강조하는 것만으로는 부족했다.\n 무엇인가 새로운 자극을 주는 대상이 필요한 것이다.\n 그래서 나폴레온은 추출된 스노우볼을 이용한다.\n 이제 스노우볼은 모든 동물들로부터 증오의 표적이 된다.\n 농장에서의 실패는 모두 그의 탓으로 돌려진다.\n 심지어는 충성도가 떨어지는 동물들은 모두 스노우볼의 앞잡이로 처단된다."
# ✅ 입력 문장을 토큰화
input_ids = tokenizer(test_text, return_tensors="pt", max_length=1024, truncation=True).input_ids

# ✅ 모델로 요약 생성
summary_ids = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    early_stopping=True,
    temperature=0.5)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("🔹 원문: ", test_text)
print("🔹 요약 결과: ", summary_text)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🔹 원문:  이상주의자인 스노우볼과 음흉한 현실주의자인 나폴레온의 싸움은 당연히 나폴레온의 승리로 끝나고 만다.
 나폴레온의 독재가 시작되었고, 이제는 철통 같은 규율만을 강조하는 것만으로는 부족했다.
 무엇인가 새로운 자극을 주는 대상이 필요한 것이다.
 그래서 나폴레온은 추출된 스노우볼을 이용한다.
 이제 스노우볼은 모든 동물들로부터 증오의 표적이 된다.
 농장에서의 실패는 모두 그의 탓으로 돌려진다.
 심지어는 충성도가 떨어지는 동물들은 모두 스노우볼의 앞잡이로 처단된다.
🔹 요약 결과:  이상주의자인 스노우볼과 음흉한 현실주의자인 나폴레온의 싸움은 결국 나폴레온의 승리로 끝이 나지만, 철통 같은 규율만을 강조하는 것만으로는 부족하고, 새로운 자극을 주는 대상이 필요한 상황이다.


In [ ]:
### 
## summary "" 로 작업용 키값을 설정했더니 줄거리가 없다고 인식하는 경우가 많았음